In [1]:
%load_ext autoreload
%autoreload 2

import os, time, pickle
from scripts.tools.issuetracker import IssuetrackerAPI
from scripts.tools.issuetracker.issues import Issues

In [2]:
def expired(loaded, expires):
    now = time.time()
    age = (now-loaded)/3600
    return age > expires

class IssueCache():
    def __init__(self, issues=None, dltime=None, expires=12, api=None):
        self.issues = None
        self._parse_issues(issues, api)
        self.loaded = dltime or time.time()
        self.expires = expires
        
    def _parse_issues(self, issues, api):
        if isinstance(issues, dict):
            self.issues = Issues(api, list(issues.values()))
        elif issues:
            self.issues = Issues(api, list(issues))
            
    def age(self):
        return self._age(self.loaded)
    
    def get(self):
        return self.issues.find()  # copies all issues
    
    @classmethod
    def from_dl(cls, api, *a, expires=12, **k):
        s = cls(expires=expires)
        s.reload(api, *a, **k)
        return s
    
    def reload(self, api, *a, **k):
        iss = api.download_issues(*a, **k)
        self._parse_issues(iss, api)
        self.loaded = time.time()
    
    def _age(self, t):
        now = time.time()
        hrs = (now - t)/3600
        return hrs        
    
    def expired(self):
        return self.age() > self.expires
    
    def save(self, fn):
        with open(fn, 'wb') as f:
            pickle.dump((self.loaded, self.issues), f)
            
    @classmethod
    def open(cls, fn, api, *a, expires=12, **k):
        try:
            s = cls.load(fn, expires)
        except (FileNotFoundError, EOFError, pickle.PickleError):
            s = cls(None, -1)  # trigger expired
            
        if s.expired():
            s.reload(api, *a, **k)
            s.save(fn)
        return s
        
    @classmethod
    def load(cls, fn, expires=12):
        with open(fn, 'rb') as f:
            ob = pickle.load(f)
        loaded, issues = ob
        return cls(issues, loaded, expires)     

In [3]:
it = IssuetrackerAPI("issue.pbsbiotech.com", "nstarkweather@pbsbiotech.com", "kookychemist")
sf = "./.issuetracker/issue_cache.pkl"

def refresh(sf=sf, it=it):
    global c, issues
    c = IssueCache.from_dl(it, "pbssoftware")
    c.save(sf)
    issues = Issues(it, c.get())
    return c.get()

if not os.path.exists(sf):
    os.makedirs(os.path.dirname(sf), exist_ok=True)
c = IssueCache.open(sf, it, "pbssoftware")
iss = c.get()
issues = Issues(it, iss)
v3_iss = issues.find2(sprint_milestone="3.0")

In [18]:
refresh()

<Issues Object: 497 Issues>

In [19]:
from jpnotebooks.Software.frs_tools import extract, matchers
from jpnotebooks.Software.frs_tools.extract import *

In [20]:
import sys
class ReleaseNotesFormatter():
    def __init__(self, issues, header=True):
        self.issues = issues
        self.release_notes = []
        self.sections = []
        self.include_header = header
        self.parse()
        
    def check(self, **kw):
        if not self.sections:
            self._extract()
        
        def match(i):
            for k,v in kw.items():
                if isinstance(v, tuple):
                    if getattr(i, k) not in v:
                        return False
                else:
                    if getattr(i, k) != v:
                        return False
            return True
        
        iset = {i.id for i in self.issues if match(i)}
        iset2 = {t[0].id for t in self.sections}
        diff = iset - iset2
        if diff:
            print("Issues without Release Notes:")
            ldiff = sorted(diff)
            for i in ldiff:
                print(" ", i)
        return ldiff
        
    def _extract(self):
        extractor = extract.IssuesExtractor(self.issues)
        self.sections = extractor.extract("Release Notes")
        
    def parse(self):
        self._extract()
        self.release_notes = rnl = []
        for issue, notes in self.sections:
            if notes.lower() == "* n/a":
                continue
            if self.include_header:
                rns = "Issue #%d - %s - %s:\n%s"%(issue.id, issue.tracker, issue.subject, notes)
            else:
                rns = notes
            rnl.append(rns)
            
    def write(self, b):
        b.write("\n\n".join(self.release_notes))
        
    def write2(self, fn):
        with open(fn, 'w') as f:
            self.write(f)
        
    def print(self):
        self.write(sys.stdout)
        
    def parse2(self):
        self.parse()
        self.print()

In [21]:
for i in issues:
    print(i.id, i.subject)

3072 Web UI Get Report Server Call
3073 3.0 getStatic call
3074 3.0 getMainValues and Advanced view
3075 3.0 Dashboard and Graphs
3077 3.0 Web UI Granular Permissions
3078 3.0 Integrity Test
3079 3.0 Export data menu
3080 3.0 Alarms
3081 3.0 Miscellaneous
989 Login Behavior
3083 Database Events.dat not being read and sent
3087 Changes to Get Alarms.vi (including bug discovered via unit testing)
2392 Restructure Alarm and Email Settings Configuration
3090 Get Sensor States for Web UI.vi and "locked" (discovered via unit testing)
1043 Make report file names more clear
3093 Error message typo in Get Sensor Calculated Value.vi (discovered via unit testing)
1048 Make page name bioreactor name
1316 change tab "Graph" to "Graphs"
2735 https encryption
3101 pH cal global clusters need order changed (bug discovered via unit testing)
3102 Get Sensor Calculated Value.vi Default String mod (discovered via unit testing)
3106 Check for Recurrent Calls.vi should set Last Access for Cookie ID, not fir

In [22]:
formatter = ReleaseNotesFormatter(issues.find2(sprint_milestone="3.0"), False)
formatter.print()

Duplicate Key Warning ('FRS1234')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Specification')
Duplicate Key Warning ('Motivation')
Got 5 warning(s) while parsing issue #3287

* The popup that appears after generating a report has been redesigned:
** The report filename is no longer a hyperlink, and does not open the report in a new tab.
** An Open Report button now opens the report in a new overlay that shows report contents.
** For remote clients, a download button prompts the user to download the file to their own computer. 
* The Get Report popup will now function correctly when Hello is in either windowed or fullscreen mode. 

* Misc. network and performance optimizations. 

* The Secondary Heat button has been removed for all reactor models and sizes. 
* Timeline selection for the Hello UI's trendline graphs has been reverted to individual button selectors for 15m, 2h, 12h, 24h, 72h, and 7d. 
* Present Value display of Le

In [23]:
unchecked = formatter.check()

Issues without Release Notes:
  2230
  2460
  2469
  2500
  3177
  3178
  3183
  3186
  3188
  3189
  3191
  3192
  3194
  3198
  3199
  3200
  3201
  3202
  3203
  3204
  3205
  3206
  3207
  3209
  3210
  3211
  3212
  3213
  3214
  3215
  3216
  3217
  3218
  3223
  3224
  3229
  3230
  3231
  3233
  3234
  3235
  3236
  3237
  3240
  3246
  3247
  3264
  3267
  3287
  3416
  3741


In [24]:
for m in unchecked:
    url = "https://issue.pbsbiotech.com/issues/%s"%str(m)
    print("Issue %s: %s"%(m,url))

Issue 2230: https://issue.pbsbiotech.com/issues/2230
Issue 2460: https://issue.pbsbiotech.com/issues/2460
Issue 2469: https://issue.pbsbiotech.com/issues/2469
Issue 2500: https://issue.pbsbiotech.com/issues/2500
Issue 3177: https://issue.pbsbiotech.com/issues/3177
Issue 3178: https://issue.pbsbiotech.com/issues/3178
Issue 3183: https://issue.pbsbiotech.com/issues/3183
Issue 3186: https://issue.pbsbiotech.com/issues/3186
Issue 3188: https://issue.pbsbiotech.com/issues/3188
Issue 3189: https://issue.pbsbiotech.com/issues/3189
Issue 3191: https://issue.pbsbiotech.com/issues/3191
Issue 3192: https://issue.pbsbiotech.com/issues/3192
Issue 3194: https://issue.pbsbiotech.com/issues/3194
Issue 3198: https://issue.pbsbiotech.com/issues/3198
Issue 3199: https://issue.pbsbiotech.com/issues/3199
Issue 3200: https://issue.pbsbiotech.com/issues/3200
Issue 3201: https://issue.pbsbiotech.com/issues/3201
Issue 3202: https://issue.pbsbiotech.com/issues/3202
Issue 3203: https://issue.pbsbiotech.com/issue

In [25]:
import time
import webbrowser
for i, m in enumerate(unchecked, 1):
    url = "https://issue.pbsbiotech.com/issues/%s"%str(m)
    if not i%5: input("Press enter to continue")
    time.sleep(0.5)
    webbrowser.open_new_tab(url)

Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue
Press enter to continue


In [236]:
from scripts.software_frs.frs_traceability2 import load_frs_from_issuetracker, build_frs_tree
items = load_frs_from_issuetracker()
reqs= build_frs_tree(items)

In [271]:
refresh()

In [274]:
webfrs_match = re.compile(r"^[-\*\+]+(3\.0WebFRS\d+)\:?[\*\+]+").match
def sort_webfrs(s):
    return int(s[-3:])
def webf(d):
    frs = []
    for l in d.splitlines():
        m = webfrs_match(l)
        if m:
            frs.append(m.group(1))
    return sorted(frs, key=sort_webfrs)

cmp = []
for i in issues:
    frs = webf(i.description)
    cmp.extend(frs)
    if frs:
        print(i.id)
        for f in frs:
            print("", f)
cmp.sort(key=sort_webfrs)

3057
 3.0WebFRS121
3066
 3.0WebFRS001
 3.0WebFRS002
 3.0WebFRS119
3067
 3.0WebFRS003
3071
 3.0WebFRS004
 3.0WebFRS005
 3.0WebFRS006
3072
 3.0WebFRS007
 3.0WebFRS008
 3.0WebFRS009
 3.0WebFRS010
 3.0WebFRS011
 3.0WebFRS012
 3.0WebFRS013
 3.0WebFRS014
3073
 3.0WebFRS015
 3.0WebFRS016
 3.0WebFRS017
 3.0WebFRS018
 3.0WebFRS019
 3.0WebFRS020
 3.0WebFRS021
 3.0WebFRS022
 3.0WebFRS023
 3.0WebFRS024
 3.0WebFRS025
 3.0WebFRS026
 3.0WebFRS027
 3.0WebFRS028
 3.0WebFRS029
 3.0WebFRS030
 3.0WebFRS031
 3.0WebFRS032
 3.0WebFRS033
 3.0WebFRS034
 3.0WebFRS035
 3.0WebFRS036
 3.0WebFRS037
 3.0WebFRS038
 3.0WebFRS039
 3.0WebFRS040
 3.0WebFRS041
 3.0WebFRS042
 3.0WebFRS043
 3.0WebFRS044
 3.0WebFRS045
 3.0WebFRS046
 3.0WebFRS047
 3.0WebFRS048
 3.0WebFRS049
 3.0WebFRS050
 3.0WebFRS051
 3.0WebFRS052
 3.0WebFRS053
 3.0WebFRS054
 3.0WebFRS055
 3.0WebFRS056
 3.0WebFRS057
 3.0WebFRS058
 3.0WebFRS059
 3.0WebFRS060
 3.0WebFRS061
 3.0WebFRS062
 3.0WebFRS063
 3.0WebFRS064
 3.0WebFRS065
 3.0WebFRS066
 3.0WebFRS067
 3.0

In [275]:
{"3.0WebFRS%03d"%i for i in range(1,122)} - set(cmp)

set()

In [278]:
for i in issues:
    if "3.0Web" in i.description:
        print("==========================")
        print("---Issue: %d---"%i.id)
        print("==========================")

---Issue: 3057---
---Issue: 3066---
---Issue: 3067---
---Issue: 3071---
---Issue: 3072---
---Issue: 3073---
---Issue: 3074---
---Issue: 3075---
---Issue: 3077---
---Issue: 3078---
---Issue: 3079---
---Issue: 3080---
---Issue: 3081---
---Issue: 3308---


In [261]:
from clipboard import *
s = paste()
lz = []
for line in s.splitlines():
    if line.startswith("3.0Web"):
        a,b = line.split(" ", 1)
        line = "\n*+" + a + "+* " + b
    lz.append(line)
copy("\n".join(lz))